In [ ]:
##MAKE SURE FILE HAS 8k CELLS and lots of genes (either 3000 most variable or 17k)

import anndata

# Load the .h5ad file
realfile = '/Users/aribahuda/Downloads/annotated_real8k.h5ad'
activafile = '/Users/aribahuda/Downloads/annotated_activa8k.h5ad'
real = anndata.read_h5ad(realfile)
activa = anndata.read_h5ad(activafile)


print(real.X.shape)
print(activa.X.shape)

In [ ]:
##Preprocessing / FILTER TO ONLY HIGHLY VARIABLE GENES (3k)

import scanpy as sc

sc.pp.filter_cells(real, min_genes=200)
sc.pp.filter_genes(real, min_cells=3)
real.var['mt'] = real.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(real, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pp.normalize_total(real, target_sum=1e4)
sc.pp.log1p(real)
sc.pp.highly_variable_genes(real, n_top_genes=3000, flavor='cell_ranger', inplace=True)
real = real[:, real.var.highly_variable]
sc.pp.regress_out(real, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(real, max_value=10)
sc.pp.neighbors(real, n_neighbors=5, n_pcs=8)





sc.pp.filter_cells(activa, min_genes=200)
sc.pp.filter_genes(activa, min_cells=3)
activa.var['mt'] = activa.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(activa, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pp.normalize_total(activa, target_sum=1e4)
sc.pp.log1p(activa)
sc.pp.highly_variable_genes(activa, n_top_genes=3000, flavor='cell_ranger', inplace=True)
adata = activa[:, activa.var.highly_variable]
sc.pp.regress_out(activa, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(activa, max_value=10)
sc.pp.neighbors(activa, n_neighbors=5, n_pcs=8)


##Double check
print(real.X.shape)
print(activa.X.shape)

In [ ]:
##Add Cell Type Annotations

if activa.shape[0] != real.shape[0]:
    raise ValueError("Number of observations (rows) in 'activa' and 'real' must be the same.")

# Set row names of 'activa' to be the same as 'real'
activa.obs_names = real.obs_names

import pandas as pd

# Reading in the original dataset labels
ann68kpbmc = pd.read_csv("https://raw.githubusercontent.com/10XGenomics/single-cell-3prime-paper/master/pbmc68k_analysis/68k_pbmc_barcodes_annotation.tsv", sep='\t')


real_val_barcodes = real.obs_names.to_frame(index=False).rename(columns={real.obs_names.name: 'Barcodes'})['Barcodes'].tolist()
real_validation_celltype = ann68kpbmc[ann68kpbmc['barcodes'].isin(real_val_barcodes)].copy()

activa_val_barcodes = activa.obs_names.to_frame(index=False).rename(columns={activa.obs_names.name: 'Barcodes'})['Barcodes'].tolist()
activa_validation_celltype = ann68kpbmc[ann68kpbmc['barcodes'].isin(activa_val_barcodes)].copy()
#print(real.obs)

import numpy as np

# Assuming 'real' and 'activa' are your AnnData objects

# Generate dataset labels for the metadata
real_labs = np.full_like(real.obs_names, fill_value="Real (Test Set)")
activa_labs = np.full_like(activa.obs_names, fill_value="ACTIVA")

# Adding dataset label to metadata
real.obs['dataset_label'] = pd.Categorical(real_labs)

# Adding Celltype to metadata
real.obs['celltype'] = pd.Categorical(real_validation_celltype['celltype'])

# ACTIVA data

# Adding dataset label to metadata
activa.obs['dataset_label'] = pd.Categorical(activa_labs)

# Adding Celltype to metadata
activa.obs['celltype'] = pd.Categorical(activa_validation_celltype['celltype'])


var_names = activa.var_names.intersection(real.var_names)
activa = activa[:, var_names]
real = real[:, var_names]

In [ ]:
##CLUSTERING + UMAP

random_state = 42
import scanpy as sc
sc.pp.pca(activa)
sc.pp.neighbors(activa, random_state=random_state)
sc.tl.leiden(activa, random_state=random_state)
sc.pl.umap(activa, color='leiden', random_state=random_state)

sc.pp.pca(real)
sc.pp.neighbors(real, random_state=random_state)
sc.tl.leiden(real, random_state=random_state)
sc.tl.umap(real)
sc.pl.umap(real, color='leiden', random_state=random_state)

print(real.obs['leiden'].value_counts())
print(activa.obs['leiden'].value_counts())

sc.tl.ingest(real, activa, obs='leiden')
sc.pl.umap(real, color=['leiden', 'celltype'], wspace=0.5)
sc.pl.umap(activa, color=['leiden', 'celltype'], wspace=0.5)

In [ ]:
##MARKER GENE DISTRIBUTION PLOTS

import matplotlib.pyplot as plt

marker_genes = ['CD79A', 'MS4A1', 'CD8A', 'CD8B', 'LYZ', 'CD14', 'LGALS3', 'S100A8',
                'GNLY', 'NKG7', 'FCGR3A', 'MS4A7', 'FCER1A', 'CST3', 'PPBP']


#sc.pl.dotplot(real, marker_genes, 'leiden', color_map='Blues')
#sc.pl.dotplot(activa, marker_genes, 'leiden')

#sc.pl.dotplot(real, marker_genes, 'celltype', vmin=0, vmax=1)
#sc.pl.dotplot(activa, marker_genes, 'celltype', vmin=0, vmax=1)


expression_values = real[:, marker_genes].X

plt.figure(figsize=(10, 6))
plt.boxplot(expression_values, vert=False, labels=marker_genes)
plt.xlabel("Expression")
plt.ylabel("Genes")
plt.xlim(-1,11)
plt.title("Marker Genes - Real")
plt.show()

expression_values = activa[:, marker_genes].X

# Step 5: Plot the gene expression values using a horizontal box and whisker plot
plt.figure(figsize=(10, 6))
plt.boxplot(expression_values, vert=False, labels=marker_genes)
plt.xlabel("Expression")
plt.xlim(-1,11)
plt.ylabel("Genes")
plt.title("Marker Genes - ACTIVA")
plt.show()

In [ ]:
##PERCENT SHARED DEGS

import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt

# Assuming 'real' and 'activa' are your Scanpy AnnData objects

# Perform differential expression analysis for both datasets
sc.tl.rank_genes_groups(real, 'leiden', method='wilcoxon')
sc.tl.rank_genes_groups(activa, 'leiden', method='wilcoxon')

# Extract top 20 differentially expressed genes for each dataset and each cluster
top_genes_real = real.uns['rank_genes_groups']['names']
top_genes_activa = activa.uns['rank_genes_groups']['names']

# Get the list of leiden clusters
leiden_clusters = real.obs['leiden'].cat.categories

# Initialize lists to store shared DEGs and total DEGs within each cluster
shared_degs_percentages = []
total_degs = []

# Iterate over each Leiden cluster
for cluster in leiden_clusters:
    # Get the top 20 DEGs for this cluster in each dataset
    top_degs_real = set(top_genes_real[cluster][:100])
    top_degs_activa = set(top_genes_activa[cluster][:100])
    
    # Calculate the intersection (shared DEGs) and union (total DEGs) of the two sets
    shared_degs = top_degs_real.intersection(top_degs_activa)
    total_degs_cluster = len(top_degs_real) + len(top_degs_activa) - len(shared_degs)
    
    # Calculate the percentage of shared DEGs within this cluster
    percentage_shared_degs = (len(shared_degs) / total_degs_cluster) * 100
    
    # Append the results to the lists
    shared_degs_percentages.append(percentage_shared_degs)
    total_degs.append(total_degs_cluster)

# Plotting
x = np.arange(len(leiden_clusters))  # x-axis for each cluster
plt.bar(x, shared_degs_percentages)
plt.xlabel('Leiden Cluster')
plt.ylabel('Percentage of Shared DEGs (%)')
plt.title('Percentage of Shared DEGs within Leiden Clusters')
plt.xticks(x, labels=leiden_clusters)
plt.show()


In [ ]:
##DEG HEATMAP

import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'real' and 'activa' are your Scanpy AnnData objects

# Perform differential expression analysis for both datasets
sc.tl.rank_genes_groups(real, 'leiden', method='wilcoxon')
sc.tl.rank_genes_groups(activa, 'leiden', method='wilcoxon')

# Get the ordered list of Leiden clusters
leiden_clusters_order = real.obs['leiden'].cat.categories.astype(int)

# Plot rank genes groups matrixplot for 'real' dataset
sc.pl.rank_genes_groups_matrixplot(real, n_genes=3,  groupby='leiden', swap_axes=True, cmap='viridis', standard_scale='var', show=None, dendrogram=False)
plt.gca().set_title('Real Dataset')
plt.xticks(ticks=range(len(leiden_clusters_order)), labels=leiden_clusters_order)

# Plot rank genes groups matrixplot for 'activa' dataset
sc.pl.rank_genes_groups_matrixplot(activa, n_genes=3, groupby='leiden', swap_axes=True, cmap='viridis', standard_scale='var', show=None, dendrogram=False)
plt.gca().set_title('Activa Dataset')
plt.xticks(ticks=range(len(leiden_clusters_order)), labels=leiden_clusters_order)

plt.tight_layout()
plt.show()


In [ ]:
##DISTRIBUTION OF VARIANCE

gene_expression_variance = real.X.var(axis=0)

# Step 2: Plot the distribution of gene expression variance
plt.figure(figsize=(8, 6))
plt.hist(gene_expression_variance, bins=50, color='skyblue', edgecolor='black')
plt.xlabel('Variance of Gene Expression')
plt.ylabel('Frequency')
plt.title('Distribution of Gene Expression Variance')
plt.show()

gene_expression_variance = activa.X.var(axis=0)

# Step 2: Plot the distribution of gene expression variance
plt.figure(figsize=(8, 6))
plt.hist(gene_expression_variance, bins=50, color='skyblue', edgecolor='black')
plt.xlabel('Variance of Gene Expression')
plt.ylabel('Frequency')
plt.title('Distribution of Gene Expression Variance')
plt.show()